In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Embedding, Concatenate, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

from sklearn.utils import shuffle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('data')
df

In [ ]:
df_user = pd.read_csv('data')
df_user

In [ ]:
df_rating = pd.read_csv('data')
df_rating

In [ ]:
df_rating.info()

In [ ]:
df_rating.isna().sum()

In [ ]:
df_rating['User-ID'] = pd.Categorical(df_rating['User-ID'])
df_rating['new_user'] = df_rating['User-ID'].cat.codes
df_rating

In [ ]:
df_rating['ISBN'] = pd.Categorical(df_rating['ISBN'])
df_rating['new_ISBN'] = df_rating['ISBN'].cat.codes
df_rating

In [ ]:
user_idx = df_rating['new_user'].values
book_idx = df_rating['new_ISBN'].values
ratings = df_rating['Book-Rating'].values

In [ ]:
N = len(set(user_idx))
M = len(set(book_idx))
K = 10

In [ ]:
user_idx, book_idx, ratings = shuffle(user_idx, book_idx, ratings)

ntrain = int(0.8 *len(ratings))

train_user = user_idx[:ntrain]
train_book = book_idx[:ntrain]
train_ratings = ratings[:ntrain]

test_user = user_idx[ntrain:]
test_book = book_idx[ntrain:]
test_ratings = ratings[ntrain:]

# ratings_avg = ratings.mean()
# train_ratings = train_ratings - ratings_avg
# test_ratings = test_ratings - ratings_avg

In [ ]:
u = Input(shape=(1,))
m = Input(shape=(1,))

u_emb = Embedding(N, K)(u)
m_emb = Embedding(M, K)(m)

u_emb = Flatten()(u_emb)
m_emb = Flatten()(m_emb)

x = Concatenate()([u_emb, m_emb])
x = Dense(1024, activation='relu')(x)
x = Dense(1)(x)

model = Model(inputs=[u, m], outputs=x)
model.compile(loss='mse', optimizer=SGD(learning_rate=0.08, momentum=0.9))

In [ ]:
r = model.fit(x=[train_user, train_book], y=train_ratings, epochs=10, batch_size=1024, validation_data=([test_user, test_book], test_ratings))

In [ ]:
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
model.save('data')